In [12]:
import json
import os

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'nba_predictions.settings')
import django
# 
django.setup()
import pandas as pd
from nba_api.stats.endpoints import playercareerstats
from nba_api.live.nba.endpoints import scoreboard
from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import \
    leaguestandingsv3, \
    leaguedashplayerstats, \
    leaguegamelog, \
    commonplayoffseries, \
    iststandings

from predictions.models import Team, Season, Player, \
    RegularSeasonStandings, \
    InSeasonTournamentStandings, PostSeasonStandings

ModuleNotFoundError: No module named 'nba_predictions.settings'

In [5]:
def fetch_ist_standings(season):
    """
    Function to fetch and update in season tournament standings
    :return:
    """
    season = Season.objects.get(slug=season)  # Assuming 'name' is the field that stores the season name
    ist_standings = {}
    ist = iststandings.ISTStandings(season=season).get_data_frames()[0]

    for team in ist['teamName'].unique():
        team_name = f"{ist.loc[ist['teamName'] == team, 'teamCity'].values[0]}" \
                    f" {ist.loc[ist['teamName'] == team, 'teamName'].values[0]}"
        team_object = Team.objects.get(name=team_name)
        team_ist_stats = {
            'ist_group': ist.loc[ist['teamName'] == team, 'istGroup'].values[0],
            'wins': ist.loc[ist['teamName'] == team, 'wins'].values[0],
            'losses': ist.loc[ist['teamName'] == team, 'losses'].values[0],
            'ist_differential': ist.loc[ist['teamName'] == team, 'diff'].values[0],
            'ist_points': ist.loc[ist['teamName'] == team, 'pts'].values[0],
            'ist_group_rank': ist.loc[ist['teamName'] == team, 'istGroupRank'].values[0],
            'ist_group_gb': ist.loc[ist['teamName'] == team, 'istGroupGb'].values[0],
            'ist_wildcard_rank': ist.loc[ist['teamName'] == team, 'istWildcardRank'].values[0],
            'ist_wildcard_gb': ist.loc[ist['teamName'] == team, 'istWildcardGb'].fillna(0).values[0],
            'ist_knockout_rank': ist.loc[ist['teamName'] == team, 'istKnockoutRank'].fillna(0).values[0]
        }
        ist_standings[team_name] = team_ist_stats
        # Update or create the InSeasonTournamentStandings object for the team
        InSeasonTournamentStandings.objects.update_or_create(
            team=team_object,
            season=season,
            season_type='ist',
            defaults=team_ist_stats
        )

    print(ist_standings)
    return ist_standings

In [6]:
fetch_ist_standings("2023-24")

NameError: name 'Season' is not defined